In [14]:
import torch
import numpy as np
import pandas as pd
import os
import copy
from tqdm import tqdm

### Load data

In [10]:
wordembedding = np.load('./origin_data/vec.npy')


In [11]:
test_word = np.load('./data/testall_word.npy')


### Add entity tag

In [4]:
def _add_entity_tag(row):
    token_sen = row['sen'].split()
    out_token_sen = copy.deepcopy(token_sen)
    update_list_e1 = []
    update_list_e2 = []
    for i, j in enumerate(token_sen):
        if j == row['e1']:
            tmp = i+len(update_list_e1)+len(update_list_e2)
            out_token_sen.insert(tmp, '[E1]')
            out_token_sen.insert(tmp+2, '[/E1]')
            
            update_list_e1.append(tmp)
            update_list_e1.append(tmp+2)
        if j == row['e2']:
            tmp = i+len(update_list_e1)+len(update_list_e2)
            update_list_e2.append(tmp)
            update_list_e2.append(tmp+2)
            out_token_sen.insert(tmp, '[E2]')
            out_token_sen.insert(tmp+2, '[/E2]')
    temp_row = copy.deepcopy(row)
    temp_row['sen'] = ' '.join(out_token_sen)
    return ' '.join(out_token_sen), temp_row

In [5]:
# Function verification
print(_add_entity_tag(full_data.iloc[0]))

('the occasion was suitably exceptional : a reunion of the 1970s-era sam rivers trio , with [E1] dave_holland [/E1] on bass and [E2] barry_altschul [/E2] on drums .', e1                                          dave_holland
e2                                        barry_altschul
rel                                                  NaN
sen    the occasion was suitably exceptional : a reun...
Name: 0, dtype: object)


In [37]:
def prepare_bert_data(dataPath):
    full_data = pd.read_csv(dataPath, header=None, sep='\t').iloc[:, 2:]
    full_data.columns = ['e1', 'e2', 'rel', 'sen']
    tagged_sen = []
    row_list = []
    with tqdm(total=len(full_data)) as pbar:
        for _, row in full_data.iterrows():
            temp_sen, temp_row = _add_entity_tag(row)
            tagged_sen.append(temp_sen)
            if len(temp_row['sen'].split())<512:
                row_list.append(temp_row)
            pbar.update(1)
    full_data.drop(columns='sen')
    full_data['seq'] = tagged_sen
    full_data = full_data.fillna(value='UNK')
    
    cleaned_df = pd.DataFrame(row_list)
    cleaned_df = cleaned_df.fillna(value='UNK')
    cleaned_df = cleaned_df.iloc[:, 2:]
    cleaned_df.to_csv(dataPath[:-4]+'_filtered.txt', index=False, sep='\t')
    full_data.to_csv(dataPath[:-4]+'_bert.txt', index=False, sep='\t')
    

In [38]:
def _clean_text(dataPath):
    output = []
    with open(dataPath, 'r') as origin_file:
        baselen = 0
        n_line = 1

        for line in origin_file.readlines():
            line = line.strip()
            token = line.split('\t')
            if baselen == 0:
                baselen = len(token)
            else:
                if len(token) != baselen:
                    print(token)
                    print(n_line)
            n_line += 1
            temp = '\t'.join(token[:6])+'\n'
            output.append(temp)
    os.rename(dataPath, dataPath[:-4]+'_original.txt')
    with open(dataPath, 'w') as outfile:
        outfile.writelines(output)

In [42]:
prepare_bert_data('./origin_data/test.txt')

100%|██████████| 172448/172448 [02:30<00:00, 1144.63it/s]


In [21]:

tagged_sen = []
with tqdm(total=len(full_data)) as pbar:
    for _, row in full_data.iterrows():
        tagged_sen.append(_add_entity_tag(row))
        pbar.update(1)

100%|██████████| 70730/70730 [00:48<00:00, 1449.71it/s]


In [24]:
print(full_data.iloc[0]['tagged'])

sen. charles e. schumer called on federal safety officials yesterday to reopen their investigation into the fatal crash of a passenger jet in [E2] belle_harbor [/E2] , [E1] queens [/E1] , because equipment failure , not pilot error , might have been the cause .


In [49]:
def convert_filter(dataPath):
    df = pd.read_csv(dataPath, sep='\t', header=None)
    df.columns=['labels', 'text']
#    df.to_json(dataPath[:-3]+'json', orient='records')
    df.to_json(dataPath[:-3]+'json')

In [50]:
convert_filter('./origin_data/train_filtered.txt')

In [24]:
def filter_long(path):
    df = pd.read_csv(path, header=None, sep='\t')
    temp = []
    for _, row in df.iterrows():
        token = row.iloc[-1].split()
        if len(token)<480:
            temp.append(row)
        else:
            print(len(token))
    print(len(temp))
    print(len(df))

In [25]:
filter_long('./origin_data/test.txt')

172448
172448


In [55]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
def convert_label(path):
    
    df = pd.read_csv(path, header=None, sep='\t')
    if not hasattr(le, 'classes_'):
        le.fit(df.iloc[:, 0])
    df.iloc[:, 0] = le.transform(df.iloc[:, 0])
    
    df.to_csv(path, header=False, index=False, sep='\t')

In [56]:
convert_label('./origin_data/train_filtered.txt')

In [51]:
print(hasattr(le, 'classes_'))

False


In [54]:
convert_label('./origin_data/test_filtered.txt')

In [9]:
from transformers import BertTokenizer
pretrain_model = "bert-base-uncased"
additional_special_tokens = ['[E1]', '[/E1]', '[E2]', '[/E2]']
tokenizer = BertTokenizer.from_pretrained(pretrain_model, do_lower_case=True, additional_special_tokens = additional_special_tokens)

Special tokens have been added in the vocabulary, make sure the associated word emebedding are fine-tuned or trained.


In [23]:

def tokenization(tokenizer, row):
    '''
    Tokenize the sentences from filter data
    '''
    sentence = '[CLS] '+row.iloc[-1]+' [SEP]'
    token = tokenizer.tokenize(sentence)
    return len(token)

def bert_token_filter(path):
    original_data = pd.read_csv(path, sep='\t', header=None)
    temp_row = []
    for _, row in tqdm(original_data.iterrows()):
        token_len = tokenization(tokenizer, row)
        if token_len>512:
            print(token_len)
        else:
            row.iloc[-1] = '[CLS] '+row.iloc[-1]+' [SEP]'
            temp_row.append(row)
    out_df = pd.DataFrame(temp_row)
    out_df.to_csv(path[:-4]+'_bf.txt', header=False, index=False, sep='\t')

In [24]:
data_path = [
    './origin_data/train_filtered.txt',
    './origin_data/test_filtered.txt'
]
for i in data_path:
    bert_token_filter(i)

38079it [00:25, 1446.80it/s]

535


39392it [00:26, 1420.66it/s]

855


41419it [00:27, 1454.56it/s]

...

56215it [00:38, 1421.62it/s]

713
785


67086it [00:46, 1429.38it/s]

855


70729it [00:48, 1448.60it/s]
172448it [02:05, 1375.69it/s]


In [19]:
original_data

,0,1
0,29,sen. charles e. schumer called on federal safe...
1,29,"but instead there was a funeral , at st. franc..."
2,29,"rosemary antonelle , the daughter of teresa l...."
3,29,one was for st. francis de sales roman catholi...
4,29,"the firefighter , whom a fire department offic..."
...,...,...
70724,53,"it gets pretty loud , '' the [E1] williams [/E..."
70725,53,"cherished grandmother of [E1] natalie [/E1] , ..."
70726,53,"cherished grandmother of [E1] natalie [/E1] , ..."
70727,53,among those present at the ceremony were phil ...
